In [147]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import pickle
import time
import sklearn
import joblib
# import gridfs
# from gridfs import GridFS
# import io
from bson.binary import Binary
from bson import ObjectId



import os
import sys
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path) 

from warehouse import DataWarehouse

module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path) 
from db.etl import MultilayerPerceptronPipeline

pipe = MultilayerPerceptronPipeline()
schema_name = 'ml__MultiLayerPerceptron'




/Users/keith/Desktop/NUS/Y2S2/IS3107/Project/ezproperty/db
/Users/keith/Desktop/NUS/Y2S2/IS3107/Project/ezproperty
Connecting to 'localhost' with user 'root'
Database | Using database is3107g6.
Pipeline | ml__MultiLayerPerceptron   | Retreiving data from data lake with _id.
Pipeline | ml__MultiLayerPerceptron   | Load success. Retrieved 1 documents.
Pipeline | ml__MultiLayerPerceptron   | Loading 1 documents to data lake.
Pipeline | ml__MultiLayerPerceptron   | Load process to data warehouse failed.


In [114]:
db = DataWarehouse()

dataset = db.query('''
    SELECT district, floorRangeStart, floorRangeEnd, area, transactionDate, resale, price FROM main__PropertyTransaction
    LIMIT 5000
''')


# Data pre-processing to convert all to float and standardise magnitude
df = pd.DataFrame(dataset)
for column in df.columns:
    if column == "transactionDate":
        df[column] = pd.to_datetime(df[column])
        df[column] = (df[column].max() - df[column]) / np.timedelta64(1,'Y')
    if column == "price": # price is in millions
        df[column] = df[column].astype(float) / 1e6
    if column == "area": # area is in 100 square feet
        df[column] = df[column].astype(float) / 100
    else:
        df[column] = df[column].astype(float)
print(df.tail())
print(df.dtypes)

Connecting to 'localhost' with user 'root'
Database | Using database is3107g6.
Database | Query executed successfully.
      district  floorRangeStart  floorRangeEnd  area  transactionDate  resale  \
4995       3.0              6.0           10.0  1.06         1.579772     0.0   
4996       3.0             16.0           20.0  0.76         1.579772     0.0   
4997       3.0             21.0           25.0  1.16         1.579772     0.0   
4998       3.0              1.0            5.0  1.16         1.579772     0.0   
4999       3.0             11.0           15.0  1.59         1.579772     0.0   

        price  
4995  2.84946  
4996  2.25270  
4997  3.30000  
4998  3.05700  
4999  4.26430  
district           float64
floorRangeStart    float64
floorRangeEnd      float64
area               float64
transactionDate    float64
resale             float64
price              float64
dtype: object


In [115]:
train = df.sample(frac=0.9,random_state=200)
test = df.drop(train.index)

X_train, Y_train = train[[column for column in df.columns if column != 'price']], train['price']
X_test, Y_test = test[[column for column in df.columns if column != 'price']], test['price']

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(4500, 6) (500, 6) (4500,) (500,)


In [116]:
# Build model
def build_mlp_model():
  model = keras.Sequential()
  model.add(Dense(6, activation='relu', input_shape=(6,)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='linear'))
  return model

mlp_model = build_mlp_model()
mlp_model.summary()

# Build model
# def build_mlp_model():
#   model = keras.Sequential()
#   model.add(Dense(6, activation='relu', input_shape=(6,)))
#   # model.add(Dropout(0.1))
#   model.add(Dense(4, activation='relu'))
#   # model.add(Dropout(0.1))
#   model.add(Dense(4, activation='relu'))
#   # model.add(Dropout(0.1))
#   model.add(Dense(1, activation='linear'))
#   return model

# mlp_model = build_mlp_model()
# mlp_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_77 (Dense)            (None, 6)                 42        
                                                                 
 dense_78 (Dense)            (None, 64)                448       
                                                                 
 dense_79 (Dense)            (None, 64)                4160      
                                                                 
 dense_80 (Dense)            (None, 64)                4160      
                                                                 
 dense_81 (Dense)            (None, 1)                 65        
                                                                 
Total params: 8,875
Trainable params: 8,875
Non-trainable params: 0
_________________________________________________________________


In [118]:
# Compile the model
mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=['mae'])
# mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
# mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [119]:
mlp_history = mlp_model.fit(X_train, Y_train, epochs=10, batch_size=16)

Epoch 1/10
282/282 [==============================] - 2s 6ms/step - loss: 0.7102 - mae: 0.7102
Epoch 2/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4813 - mae: 0.4813
Epoch 3/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4656 - mae: 0.4656
Epoch 4/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4568 - mae: 0.4568
Epoch 5/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4454 - mae: 0.4454
Epoch 6/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4348 - mae: 0.4348
Epoch 7/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4433 - mae: 0.4433
Epoch 8/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4424 - mae: 0.4424
Epoch 9/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4288 - mae: 0.4288
Epoch 10/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4272 - mae: 0.4272


In [120]:
loss, acc = mlp_model.evaluate(X_test, Y_test)
print(X_test.shape)
print(f'test loss is {loss}')
print(f'test accuracy is {acc}')

16/16 [==============================] - 0s 5ms/step - loss: 0.3825 - mae: 0.3825
(500, 6)
test loss is 0.38253888487815857
test accuracy is 0.38253888487815857


In [148]:
def save_model_to_db():

    #pickling the model
    pickled_model = pickle.dumps(mlp_model)

    # using joblib
    joblib.dump(mlp_model, "test")

    
    # pickled_model = tf.keras.models.save_model(mlp_model, 'my_model.h5')
    # model_file = 'my_model.h5'

    # with open(model_file, "rb") as f:
    #     encoded = Binary(f.read())
        

    

    # # creating other attributes
    # model = [{ "model": model_file, 'name': "MLP", 'created_time': time.time()}]
    # pipe.dl_loader(model, schema_name)
    

save_model_to_db()

In [150]:
def load_model_to_db():
    json_data = {}

    result = pipe.dl_getter('ml__MultiLayerPerceptron')
    # print(result)
    # print(result[0]["model"])
    # pickled_model = json_data[result[0]["model"]]
    model = pickle.loads(result[0]["model"])
    # print(model)

    # model = joblib.load("test")

    # pred_price = model.predict(3.0, 6.0, 10.0, 1.06, 1.579772, 0.0)

    # print(pred_price)

load_model_to_db()
 

    

AttributeError: 'Adam' object has no attribute 'build'